In [21]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
import json
import yfinance as yf
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [22]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")

# Retrieve BTC Price from Kraken API

In [23]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close']
SMA_8 = ohlc[0]['8SMA'] = ohlc[0]['close'].rolling(8).mean()
SMA_200 = ohlc[0]['200SMA'] = ohlc[0]['close'].rolling(200).mean()
EMA_200 = ohlc[0]['200EMA'] = ohlc[0]['close'].ewm(200).mean()
ta_df = ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1)
# ta_df.plot(figsize = (25,10))
ta_df.head()

,Price,8SMA,200SMA,200EMA
dtime,,,,
2020-07-02,9086.0,NaN,NaN,9086.000000
2020-07-03,9058.0,NaN,NaN,9071.965087
2020-07-04,9138.5,NaN,NaN,9094.254098
2020-07-05,9083.8,NaN,NaN,9091.620989
2020-07-06,9348.2,NaN,NaN,9143.449943


### Plot BTC Price over Time

In [24]:
BTC = ohlc[0].hvplot.line(
    x = 'dtime',
    y = 'Price',
    xlabel = 'Date',
    ylabel = 'Price',
    title = 'Lunar Lotto',
    legend = True
)
BTC

:Curve   [dtime]   (Price)

# Clean Lunar Data


In [25]:
#Create a Full Moon Dataframe
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full_Moon'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)

In [26]:
#Create a New Moon column
full_moon['New_Moon'] = full_moon['Full_Moon'] + timedelta(days=14)

In [27]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(ohlc[0]['Price'], on = 'Full_Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

full_moon = full_moon.join(ohlc[0]['Price'], on = 'New_Moon', how = 'inner')
full_moon['New Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

moon_data = full_moon.drop(columns=['index','level_0'])

moon_data.tail()

,Full_Moon,New_Moon,Full Moon Price,New Moon Price
19,2022-01-18,2022-02-01,42383.0,38719.9
20,2022-02-16,2022-03-02,43873.1,43922.5
21,2022-03-18,2022-04-01,41775.1,46312.0
22,2022-04-16,2022-04-30,40381.0,37660.8
23,2022-05-16,2022-05-30,29826.1,31711.7


# Plot Lunar Data over BTC Data

In [28]:
# create hvplot figures to then overaly 
glyph_1 = ohlc[0]['Price'].hvplot.line( 
    'dtime', 'Price',
    color='grey',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = moon_data.hvplot.scatter(
    x = 'Full_Moon',
    y = 'Full Moon Price',
    color='yellow',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = moon_data.hvplot.scatter(
    x = 'New_Moon',
    y = 'New Moon Price',
    color='purple',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_4 = ohlc[0]['8SMA'].hvplot.line(
    'dtime', '8SMA',
    color='blue',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_5 = ohlc[0]['200SMA'].hvplot.line(
    'dtime', '200SMA',
    color='green',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_6 = ohlc[0]['200EMA'].hvplot.line(
    'dtime', '200EMA',
    color='red',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_1*glyph_2*glyph_3*glyph_4*glyph_5*glyph_6

:Overlay
   .Curve.Price    :Curve   [dtime]   (Price)
   .Scatter.I      :Scatter   [Full_Moon]   (Full Moon Price)
   .Scatter.II     :Scatter   [New_Moon]   (New Moon Price)
   .Curve.A_8SMA   :Curve   [dtime]   (8SMA)
   .Curve.A_200SMA :Curve   [dtime]   (200SMA)
   .Curve.A_200EMA :Curve   [dtime]   (200EMA)

## Prepare Lunar and BTC Data for Merge

In [29]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close']
SMA_8 = ohlc[0]['8SMA'] = ohlc[0]['close'].rolling(8).mean()
SMA_200 = ohlc[0]['200SMA'] = ohlc[0]['close'].rolling(200).mean()
EMA_200 = ohlc[0]['200EMA'] = ohlc[0]['close'].ewm(200).mean()
ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1, inplace=True)
ohlc = ohlc[0]
ohlc.reset_index(inplace=True)
ohlc['dtime'] = pd.to_datetime(ohlc['dtime'])
ohlc.tail()

,dtime,Price,8SMA,200SMA,200EMA
715,2022-06-17,20432.9,23991.8250,39953.8260,40586.408206
716,2022-06-18,18950.0,22728.7125,39763.6725,40475.665180
717,2022-06-19,20555.5,21750.6250,39580.2605,40373.721115
718,2022-06-20,20550.0,20998.1750,39400.3785,40272.285070
719,2022-06-21,20831.6,20794.6125,39236.1435,40172.823085


In [30]:
# Read Full_Moon CSV, Clean index, infer Datetime
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'dtime'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)
full_moon['Phase'] = full_moon.loc['dtime',:] = 'Full Moon'
full_moon.drop(full_moon.index[-1], inplace=True)

In [31]:
# Timedelta + 14 days to create New_Moon Dataframe
new_moon = pd.DataFrame(full_moon['dtime'] + timedelta(days=14))
new_moon['Phase'] = new_moon.loc['dtime',:] = 'New Moon'
new_moon.drop(new_moon.index[-1], inplace=True)

In [32]:
# Append Lunar Dataframes to create one table sorted by 'dtime'
phase_data = full_moon.append(new_moon)
phase_data.sort_values('dtime', inplace= True)
phase_data.reset_index(inplace=True)
phase_data.drop(['index'], axis = 1, inplace=True)
phase_data['dtime'] = pd.to_datetime(phase_data['dtime'])
phase_data.columns
phase_data

,dtime,Phase
0,1900-01-15,Full Moon
1,1900-01-29,New Moon
2,1900-02-14,Full Moon
3,1900-02-28,New Moon
4,1900-03-16,Full Moon
...,...,...
3731,2050-11-13,New Moon
3732,2050-11-28,Full Moon
3733,2050-12-12,New Moon
3734,2050-12-28,Full Moon


# Merge Dataframes on 'dtime'

In [33]:
moon_merge = phase_data.merge(ohlc, on='dtime', how='inner')
moon_merge.tail()

,dtime,Phase,Price,8SMA,200SMA,200EMA
44,2022-04-16,Full Moon,40381.0,40824.1375,48164.3660,43871.100148
45,2022-04-30,New Moon,37660.8,39090.8375,47363.9175,43580.701837
46,2022-05-16,Full Moon,29826.1,29925.8375,45149.3915,42774.075271
47,2022-05-30,New Moon,31711.7,29521.3875,42799.7910,41857.278253
48,2022-06-14,Full Moon,22111.0,27492.5625,40486.6075,40888.552072


In [34]:
def FM_result(x, y):
    if x - y < 0:
        return True
    return False

moon_data['FM Result'] = moon_data.apply(lambda row: FM_result(row['Full Moon Price'], row['New Moon Price']), axis=1)
moon_data['FM Percentage Difference'] = (moon_data['Full Moon Price'] - moon_data['New Moon Price']) / moon_data['Full Moon Price']

def NM_result(x, y):
    if x - y > 0:
        return True
    return False

moon_data['NM Result'] = moon_data.apply(lambda row: NM_result(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data['NM Percentage Difference'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])


moon_data.head()

,Full_Moon,New_Moon,Full Moon Price,New Moon Price,FM Result,FM Percentage Difference,NM Result,NM Percentage Difference
0,2020-07-05,2020-07-19,9083.8,9213.7,True,-0.014300,True,0.014099
1,2020-08-03,2020-08-17,11237.3,12301.7,True,-0.094720,True,0.086525
2,2020-09-02,2020-09-16,11396.9,10951.1,False,0.039116,False,-0.040708
3,2020-10-01,2020-10-15,10616.1,11501.0,True,-0.083355,True,0.076941
4,2020-10-31,2020-11-14,13809.1,16080.1,True,-0.164457,True,0.141230


In [35]:
# Create a New Temporary Data Frame to Finalize the Moon Price Behavior Data

frame = {'Full Moon': moon_data.iloc[:,0], 'New Moon': moon_data.iloc[:,1], 'Full Moon Price': moon_data.iloc[:,2], 'New Moon Price': moon_data.iloc[:,3]} #'New Moon Price': moon_data.iloc[1:,3]}
tmpdf = pd.DataFrame(frame)
tmpdf['Difference']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'] #.shift(1)
tmpdf['Percentage Difference'] = ((tmpdf['Full Moon Price'] - tmpdf['New Moon Price']) / tmpdf['New Moon Price'])*100
tmpdf['Trade Profit']= tmpdf['Difference'].cumsum()
tmpdf['Buy and Hold']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'][0]

def returnBoolFM(x):
    if x < 0:
        return True
    return False

def returnBoolNM(x):
    if x > 0:
        return True
    return False

tmpdf['Did We Profit this Month'] = tmpdf.apply(lambda row: returnBoolNM(row['Difference']), axis = 1)


tmpdf

,Full Moon,New Moon,Full Moon Price,New Moon Price,Difference,Percentage Difference,Trade Profit,Buy and Hold,Did We Profit this Month
0,2020-07-05,2020-07-19,9083.8,9213.7,129.9,-1.409857,129.9,129.9,True
1,2020-08-03,2020-08-17,11237.3,12301.7,1064.4,-8.652463,1194.3,3217.9,True
2,2020-09-02,2020-09-16,11396.9,10951.1,-445.8,4.070824,748.5,1867.3,False
3,2020-10-01,2020-10-15,10616.1,11501.0,884.9,-7.694114,1633.4,2417.2,True
4,2020-10-31,2020-11-14,13809.1,16080.1,2271.0,-14.123046,3904.4,6996.3,True
5,2020-11-30,2020-12-14,19708.1,19281.1,-427.0,2.214604,3477.4,10197.3,False
6,2020-12-30,2021-01-13,28879.9,37390.1,8510.2,-22.760570,11987.6,28306.3,True
7,2021-01-28,2021-02-11,33430.4,47962.8,14532.4,-30.299315,26520.0,38879.0,True
8,2021-02-27,2021-03-13,46170.2,61174.9,15004.7,-24.527543,41524.7,52091.1,True
9,2021-03-28,2021-04-11,55782.0,59956.0,4174.0,-6.961772,45698.7,50872.2,True


In [36]:
#plot Difference column vs the buy and hold column
buy_hold_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Buy and Hold',
    xlabel = 'Date',
    ylabel = 'Buy and Hold',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'red',
    legend = True
)

moon_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Trade Profit',
    xlabel = 'Date',
    ylabel = 'Difference',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'blue',
    legend = True
    
)

moon_glyph*buy_hold_glyph

:Overlay
   .Curve.I  :Curve   [Full Moon]   (Trade Profit)
   .Curve.II :Curve   [Full Moon]   (Buy and Hold)

In [37]:
# Define a function to count the Trues in the 'Did We Profit this Month' column
def count_tr(x):
    return x.sum()
number_of_Trues = (count_tr(tmpdf['Did We Profit this Month']))

#Count the number of rows in the Did We Profit this Month column

number_of_Rows = tmpdf['Did We Profit this Month'].count()

# The PERCENTAGE of times you would profit if you bought on Full Moon and sold on New Moon each month

print(f'{(number_of_Trues/number_of_Rows)*100:.2f}% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!')

66.67% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!


In [38]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


In [39]:
fear_data = json.dumps(FearGreedUrl["data"][0], indent=4)
fear_json = json.loads(fear_data)
fear_json


{'value': '9',
 'value_classification': 'Extreme Fear',
 'timestamp': '1655769600',
 'time_until_update': '3741'}

In [40]:
fear_df = pd.DataFrame(data=fear_json, columns=['value', 'value_classification', 'timestamp'])

ValueError: If using all scalar values, you must pass an index